In [1]:
import os
import sys
from pathlib import Path

# Lấy đường dẫn project root (NLP-project)
PROJECT_ROOT = Path.cwd().parent

# Chuyển working directory về project root
os.chdir(PROJECT_ROOT)

# Thêm src vào PYTHONPATH để import được các file .py
sys.path.append(str(PROJECT_ROOT / "src"))

print("Working directory:", os.getcwd())
print("src in path:", PROJECT_ROOT / "src")

Working directory: d:\studyhk1-25-26\NLP\project
src in path: d:\studyhk1-25-26\NLP\project\src


In [6]:
from train import main
main()

Loading data...
SRC vocab size: 5893
TGT vocab size: 6470
Training model...

=== Epoch 1/12 | TF=0.500 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:09<00:00,  1.71it/s]


Epoch 01/12 | Train Loss: 5.007 | Val Loss: 4.462 | Gap: 0.545 | Val BLEU: 2.64 | Time: 1173.4s
  → Saved best checkpoint (by Val BLEU) to: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth

=== Epoch 2/12 | TF=0.485 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:12<00:00,  1.32it/s]


Epoch 02/12 | Train Loss: 4.066 | Val Loss: 3.886 | Gap: 0.180 | Val BLEU: 7.59 | Time: 1266.9s
  → Saved best checkpoint (by Val BLEU) to: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth

=== Epoch 3/12 | TF=0.470 | LR=0.000700 ===


...

Epoch 10/12 | Train Loss: 2.308 | Val Loss: 2.946 | Gap: 0.638 | Val BLEU: 23.72 | Time: 1324.2s
  → Saved best checkpoint (by Val BLEU) to: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth

=== Epoch 11/12 | TF=0.369 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:10<00:00,  1.47it/s]


Epoch 11/12 | Train Loss: 2.228 | Val Loss: 2.918 | Gap: 0.690 | Val BLEU: 23.97 | Time: 1327.4s
  → Saved best checkpoint (by Val BLEU) to: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth

=== Epoch 12/12 | TF=0.358 | LR=0.000700 ===


generate: 100%|██████████| 16/16 [00:10<00:00,  1.55it/s]


Epoch 12/12 | Train Loss: 2.162 | Val Loss: 2.919 | Gap: 0.757 | Val BLEU: 24.39 | Time: 1325.9s
  → Saved best checkpoint (by Val BLEU) to: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth
Saved metrics to: D:\studyhk1-25-26\NLP\project\results\metrics.json

Training finished.
Best Val Loss (for report) = 2.918
Best Val BLEU (saved ckpt) = 24.39


In [7]:
from evaluate import run_evaluate

run_evaluate()

Loading checkpoint: D:\studyhk1-25-26\NLP\project\checkpoints\best_model.pth
→ Detected model type: Seq2SeqWithAttention (Luong)
Generating hypotheses on test set...


generate: 100%|██████████| 17/17 [00:08<00:00,  2.03it/s]

Computing BLEU on test set...
Test BLEU: 13.87869996913708
Saving sample translations to: D:\studyhk1-25-26\NLP\project\results\samples.txt


13.87869996913708

In [8]:
from infer import translate

translate("A man is riding a bike")

'homme fait du vélo'